In [1]:
%reload_ext autoreload
%autoreload 2

from src.acnets.deep.lemon_data import LEMONDataModule
import numpy as np
import torch
from torch import nn
import torch.nn.functional  as F
import pywt
import matplotlib.pyplot as plt
import seaborn as sns

from src.acnets.deep.cvae import CVAE

In [2]:
datamodule = LEMONDataModule(n_subjects=8, test_ratio=None)
datamodule.setup()

batch = datamodule.train[:]
x_wavelets = batch[5] # -> shape: (subjects, wavelets, regions)

n_channels = x_wavelets.shape[2]
n_wavelets = 32
n_embeddings = 16

x = x_wavelets[:, :n_wavelets, :].transpose(1, 2)  # -> shape: (subjects, regions, wavelets)

h, x_recon, loss = CVAE(n_channels, n_embeddings)(x)
x.shape, h.shape, x_recon.shape, loss


(torch.Size([8, 160, 32]),
 torch.Size([8, 16]),
 torch.Size([8, 160, 32]),
 tensor(3.6385, grad_fn=<AddBackward0>))

In [3]:
# # DEBUG PLOTS
# plt.subplots(1, 2, figsize=(10, 3))
# plt.subplot(1, 2, 1)
# plt.title('original')
# sns.heatmap(ts_regions[0].T, cmap='viridis')

# # plot 100 wavelet coefficients of the first subject
# plt.subplot(1, 2, 2)
# plt.title('coefs')
# sns.heatmap(wt_regions[0][:100,:].T, cmap='viridis', label='coefs')
# plt.show()